In [1]:
import sqlite3
import pandas as pd

CRIM - уровень преступности на душу населения
ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
INDUS - доля акров неторгового бизнеса на город.
CHAS - 1, если граничит с рекой, 0 иначе
NOX - концентрация оксидов азота (частей на 10 миллионов)
RM - среднее количество комнат в квартире
AGE - доля жилых домов, построенных до 1940 г.
DIS - взвешенные расстояния до пяти центров занятости Бостона
RAD - индекс доступности к радиальным магистралям
TAX - полная ставка налога на имущество за 10 000 долларов США
PTRATIO - соотношение учеников и учителей по городам
B - 1000(Bk - 0.63)^2, где Bk доля чернокожих людей по городам
LSTAT - % более низкого статуса населения
MEDV - Средняя стоимость домов в 1000 долларов.

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [6]:
pd.read_sql(
    """
    SELECT 'CRIM' AS 'variable', SUM(CASE WHEN CRIM IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'ZN', SUM(CASE WHEN ZN IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'INDUS', SUM(CASE WHEN INDUS IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'CHAS', SUM(CASE WHEN CHAS IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'NOX', SUM(CASE WHEN NOX IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston;
    """,
    con,
)

,variable,nulls_count
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [12]:
pd.read_sql(
    """
    SELECT 'CRIM' AS 'col', COUNT(CRIM) AS nunique
      FROM (SELECT DISTINCT CRIM FROM boston)
     UNION ALL
    SELECT 'ZN', COUNT(ZN) as nunique
      FROM (SELECT DISTINCT ZN FROM boston)
     UNION ALL
    SELECT 'INDUS', COUNT(INDUS) AS nunique
      FROM (SELECT DISTINCT INDUS FROM boston)
     UNION ALL
    SELECT 'CHAS', COUNT(CHAS) AS nunique
      FROM (SELECT DISTINCT CHAS FROM boston)
     UNION ALL
    SELECT 'NOX', COUNT(NOX) AS nunique
      FROM (SELECT DISTINCT NOX FROM boston);
    """,
    con,
)

,col,nunique
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [15]:
pd.read_sql(
    '''
  SELECT COLUMN_NAME, MEDIANA, MIN FROM(
    SELECT * FROM(
    SELECT 'ZN' AS COLUMN_NAME, ZN as MEDIANA, MIN(ZN) OVER(ORDER BY ZN) as MIN, row_number() OVER(ORDER BY ZN) as ROW
    FROM boston)
    WHERE ROW = 506 / 2
    
    UNION ALL
    SELECT * FROM(
    SELECT 'CRIM', CRIM, MIN(CRIM) OVER(ORDER BY CRIM), row_number() OVER(ORDER BY CRIM) as ROW
    FROM boston)
    WHERE ROW = 506 / 2
    
    UNION ALL
    SELECT * FROM(
    SELECT 'INDUS', INDUS, MIN(INDUS) OVER(ORDER BY INDUS), row_number() OVER(ORDER BY INDUS) AS ROW
    FROM boston)
    WHERE ROW = 506 / 2
    
    UNION ALL
    SELECT * FROM(
    SELECT 'CHAS', CHAS, MIN(CHAS) OVER(ORDER BY CHAS), row_number() OVER(ORDER BY CHAS) AS ROW
    FROM boston)
    WHERE ROW = 506 / 2
    
    UNION ALL
    SELECT * FROM(
    SELECT 'NOX', NOX, MIN(NOX) OVER(ORDER BY NOX), row_number() OVER(ORDER BY NOX) AS ROW
    FROM boston)
    WHERE ROW = 506 / 2
    )
    WHERE MEDIANA = MIN
    ''',
    con,
)

,COLUMN_NAME,MEDIANA,MIN
0,ZN,0.0,0.0
1,CHAS,0.0,0.0


Вывод:Большая часть значений переменных ZN и CHAS соответствует их минимальным значениям. Так как CHAS является бинарной переменной, это означает, что большинство домов расположены не рядом с рекой. Что касается ZN, похоже, что большинство домов не располагаются на участках, предназначенных для застройки..

4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [16]:
pd.read_sql(
    """
    SELECT *
      FROM (SELECT LSTAT, MEDV, rank() OVER (ORDER BY LSTAT DESC) AS rank_by_LSTAT
              FROM boston)
     WHERE MEDV = (SELECT MAX(MEDV) FROM boston);
    """,
    con,
)

,LSTAT,MEDV,rank_by_LSTAT
0,9.53,50.0,304
1,8.88,50.0,326
2,7.44,50.0,365
3,5.12,50.0,440
4,4.63,50.0,457
5,4.45,50.0,465
6,3.73,50.0,480
7,3.70,50.0,481
8,3.32,50.0,489
9,3.26,50.0,490


Вывод:В дорогих домах проживают представители обеспеченных семей..

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [17]:
pd.read_sql(
    """
      SELECT CASE
                WHEN CHAS = 1 THEN 'Yes'
                ELSE 'No'
             END AS near_river,
             AVG(MEDV) AS avg_price_kUSD
        FROM boston
    GROUP BY CHAS;
    """,
    con,
)

,near_river,avg_price_kUSD
0,No,22.093843
1,Yes,28.440000


В среднем около реки дороже

6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [18]:
pd.read_sql(
    '''
    SELECT * FROM(
    SELECT 'CRIM' as 'Name_col', (SELECT AVG(CRIM) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    
    UNION ALL
    SELECT 'ZN' as 'Name_col', (SELECT AVG(ZN) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    
    UNION ALL
    SELECT 'INDUS' as 'Name_col', (SELECT AVG(INDUS) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    
    UNION ALL
    SELECT 'NOX' as 'Name_col', (SELECT AVG(NOX) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    )
    WHERE CHAS_0 < CHAS_1
    ''',
    con,
)

,Name_col,CHAS_0,CHAS_1
0,INDUS,11.019193,12.719143
1,NOX,0.551817,0.593426


Как видно, возле реки расположено больше промышленных заводов, что приводит к более высокому уровню диоксида азота в атмосфере.

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [20]:
pd.read_sql(
    """
    SELECT n.perc_name, indus_val, nox_val
      FROM (SELECT perc_name * 10 AS perc_name, indus_val
              FROM (SELECT perc_name,
                           LAST_VALUE(INDUS) OVER(PARTITION BY perc_name) AS indus_val
                      FROM (SELECT INDUS,
                                   NTILE(10) OVER(ORDER BY INDUS) AS perc_name
                              FROM boston))
          GROUP BY indus_val) AS n
      JOIN (SELECT perc_name * 10 AS perc_name, nox_val
              FROM (SELECT perc_name,
                           LAST_VALUE(NOX) OVER(PARTITION BY perc_name) AS nox_val
                      FROM (SELECT NOX,
                                   NTILE(10) OVER(ORDER BY NOX) AS perc_name
                              FROM boston))
          GROUP BY nox_val) AS i
        ON n.perc_name = i.perc_name;
    """,
    con,
)

,perc_name,indus_val,nox_val
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,13.89,0.580
6,70,18.10,0.609
7,90,19.58,0.713
8,100,27.74,0.871


Наблюдается прямая зависимость между уровнем промышленной застройки и концентрацией диоксида азота в воздухе.